In [2]:
!pip install dask\[complete\]

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 48.3 MB/s eta 0:00:00m eta 0:00:010:01:01
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849766 sha256=cd02ab18f40e5d7b12d50303a959af853d78d8c91d53396b28d1b02a08236b2f
  Stored in directory: /Users/ritayanpatra/Library/Caches/pip/wheels/13/92/64/da92a3521323cc629fdf25dd56eb26938e08014c1b57ad3759
Successfully built pyspark


In [4]:
!git clone https://github.com/dgadiraju/data

Cloning into 'data'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 156 (delta 3), reused 7 (delta 1), pack-reused 145 (from 1)
Receiving objects: 100% (156/156), 282.65 MiB | 32.13 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Updating files: 100% (78/78), done.


In [5]:
!ls -ltr data/nyse_all/nyse_data

total 272040
-rw-r--r--@ 1 ritayanpatra  staff   3842443 Jan 20 23:51 NYSE_1997.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   4142942 Jan 20 23:51 NYSE_1998.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   4297025 Jan 20 23:51 NYSE_1999.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   4439306 Jan 20 23:51 NYSE_2000.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   4722623 Jan 20 23:51 NYSE_2001.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   5021940 Jan 20 23:51 NYSE_2002.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   5271305 Jan 20 23:51 NYSE_2003.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   5689069 Jan 20 23:51 NYSE_2004.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   6207833 Jan 20 23:51 NYSE_2005.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   6480175 Jan 20 23:51 NYSE_2006.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   6903056 Jan 20 23:51 NYSE_2007.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   7179621 Jan 20 23:51 NYSE_2008.txt.gz
-rw-r--r--@ 1 ritayanpatra  staff   7186235 Jan 20 23:51 NYSE_2009.txt.gz
-rw-r--r--@ 1 ritayanpatr